In [1]:
from flow.networks import Network
from flow.core.params import VehicleParams,InFlows,SumoCarFollowingParams,SumoParams, EnvParams, InitialConfig, NetParams, SumoLaneChangeParams
from flow.controllers import IDMController, RLController
from flow.envs import TestEnv
from flow.controllers.base_routing_controller import BaseRouter
import numpy as np
import time

In [2]:
from routes_generation import *

In [3]:
#####Configurations#####
DEBUG = True
#DEBUG = False

TRAINING = True
#TRAINING = False

#RENDER = False
RENDER = True

MODEL = 'gat'

RAY_RL = False

In [4]:
# %run 2-routes_generate.ipynb

# #%run KSP2_DiGraph_node.ipynb

In [5]:
def clean_nested(l):
    cleaned = []
    for v in l:
        if isinstance(v, list):
            v = clean_nested(v)
            if not v:
                continue
        cleaned.append(v)
    return cleaned

In [6]:
def flatten(x):
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

In [7]:
Net_Dir = "/Users/raynedu/baseDir"

In [8]:
# create some default parameters parameters
env_params = EnvParams()
sim_params = SumoParams(render=RENDER, sim_step=1, restart_instance=True)
initial_config = InitialConfig()
# VEH_COLORS = ['red','red']
vehicles = VehicleParams()

In [9]:
import os

net_params = NetParams(
    template=os.path.join(Net_Dir, "Manhattan_highway_shrink.net.xml"),
    
)

In [10]:
new_initial_config = InitialConfig(
    edges_distribution=EDGES_DISTRIBUTION,#['46522160#0','198595461#0','gneE127','458166897#10','420496091#0','gneE180'],
    bunching = 40,
    spacing = "random"
)

In [11]:
class RayneNetwork(Network):
    def specify_routes(self, net_params):
        rts = {}
        for i in range(len(EDGES_DISTRIBUTION)):
            group_EDGES = []
            for j in range(len(EDGES)):
                if EDGES[j][0][0] == EDGES_DISTRIBUTION[i]:
                    group_EDGES.append(EDGES[j])
            rts[EDGES_DISTRIBUTION[i]] = group_EDGES
        #update the specify routes based on the weight   
#         for i in range(1,9):
#             if env.step_counter == i*200:       
        return rts

In [12]:
#EDGES

In [13]:
#Rayne's specific environment
import numpy as np
from gym.spaces.box import Box
from gym.spaces import Tuple

class RayneEnv(TestEnv):  # update my environment class
    #print(step_counter)
    def observation_space(self):
        N = 6
        F = 2
        states = Box(low = -np.inf, high = np.inf, shape = (N,F), dtype = np.float32)
        adjacency = Box(low = 0, high = 1, shape = (N,N), dtype = np.int32)
        
        return Tuple([states, adjacency])
    
    def action_space(self):
        N = 6
        return Box(low = 0, high = 1, shape = (N,), dtype = np.int32)
                
    def get_state(self):
        if self.done == True:
            #print(self.step_counter)
            #print(self.k.vehicle.get_route('flow_20123.1'))
            #generate features for GAT
            #six fog nodes features: average vehicle speed and V/C
            Average_Vehicle_Speed = []
            Average_V_C = []
            #Node_A
            ids = self.k.vehicle.get_ids()
            Vehicle_in_A = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogA_Edges]# vehicles on different edges
            Vehicle_ID_Edge_A = flatten(clean_nested(Vehicle_in_A))
            Vehicle_Speed_A = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_A]
            Average_Vehicle_Speed_A = np.mean(Vehicle_Speed_A)

            #get lanes number and length of the edges
            V_C_A = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogA_Edges]
            V_C_A = [i for i in V_C_A if i != 0]
            Average_V_C_A = np.mean(V_C_A)

            if Average_V_C_A > 5.0 and Average_Vehicle_Speed_A == 0.0:
                Average_Vehicle_Speed_A = 8
            if Average_V_C_A > 0.0 and Average_V_C_A <= 5.0 and Average_Vehicle_Speed_A == 0.0:
                Average_Vehicle_Speed_A = 3
            #print(Average_Vehicle_Speed_A, Average_V_C_A)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_A)
            Average_V_C.append(Average_V_C_A)
            #Node_B
            ids = self.k.vehicle.get_ids()
            Vehicle_in_B = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogB_Edges]# vehicles on different edges
            Vehicle_ID_Edge_B = flatten(clean_nested(Vehicle_in_B))
            Vehicle_Speed_B = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_B]
            Average_Vehicle_Speed_B = np.mean(Vehicle_Speed_B)

            #get lanes number and length of the edges
            V_C_B = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogB_Edges]
            V_C_B = [i for i in V_C_B if i != 0]
            Average_V_C_B = np.mean(V_C_B)

            if Average_V_C_B > 5.0 and Average_Vehicle_Speed_B == 0.0:
                Average_Vehicle_Speed_B = 8
            if Average_V_C_B > 0.0 and Average_V_C_B <= 5.0 and Average_Vehicle_Speed_B == 0.0:
                Average_Vehicle_Speed_B = 3
            #print(Average_Vehicle_Speed_B, Average_V_C_B)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_B)
            Average_V_C.append(Average_V_C_B)

            #Node_C
            ids = self.k.vehicle.get_ids()
            Vehicle_in_C = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogC_Edges]# vehicles on different edges
            Vehicle_ID_Edge_C = flatten(clean_nested(Vehicle_in_C))
            Vehicle_Speed_C = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_C]
            Average_Vehicle_Speed_C = np.mean(Vehicle_Speed_C)

            #get lanes number and length of the edges
            V_C_C = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogC_Edges]
            V_C_C = [i for i in V_C_C if i != 0]
            Average_V_C_C = np.mean(V_C_C)

            if Average_V_C_C > 5.0 and Average_Vehicle_Speed_C == 0.0:
                Average_Vehicle_Speed_C = 8.0
            if Average_V_C_C > 0.0 and Average_V_C_C <= 5.0 and Average_Vehicle_Speed_C == 0.0:
                Average_Vehicle_Speed_C = 3.0
            #print(Average_Vehicle_Speed_C, Average_V_C_C)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_C)
            Average_V_C.append(Average_V_C_C)
            #Node_D
            ids = self.k.vehicle.get_ids()
            Vehicle_in_D = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogD_Edges]# vehicles on different edges
            Vehicle_ID_Edge_D = flatten(clean_nested(Vehicle_in_D))
            Vehicle_Speed_D = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_D]
            Average_Vehicle_Speed_D = np.mean(Vehicle_Speed_D)

            #get lanes number and length of the edges
            V_C_D = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogD_Edges]
            V_C_D = [i for i in V_C_D if i != 0]
            Average_V_C_D = np.mean(V_C_D)

            if Average_V_C_D > 5.0 and Average_Vehicle_Speed_D == 0.0:
                Average_Vehicle_Speed_D = 8
            if Average_V_C_D > 0.0 and Average_V_C_D <= 5.0 and Average_Vehicle_Speed_D == 0.0:
                Average_Vehicle_Speed_D = 3
            #print(Average_Vehicle_Speed_D, Average_V_C_D)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_D)
            Average_V_C.append(Average_V_C_D)
            #Node_E
            ids = self.k.vehicle.get_ids()
            Vehicle_in_E = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogE_Edges]# vehicles on different edges
            Vehicle_ID_Edge_E = flatten(clean_nested(Vehicle_in_E))
            Vehicle_Speed_E = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_E]
            Average_Vehicle_Speed_E = np.mean(Vehicle_Speed_E)

            #get lanes number and length of the edges
            V_C_E = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogE_Edges]
            V_C_E = [i for i in V_C_E if i != 0]
            Average_V_C_E = np.mean(V_C_E)

            if Average_V_C_E > 5.0 and Average_Vehicle_Speed_E == 0.0:
                Average_Vehicle_Speed_E = 8
            if Average_V_C_E > 0.0 and Average_V_C_E <= 5.0 and Average_Vehicle_Speed_E == 0.0:
                Average_Vehicle_Speed_E = 3
            #print(Average_Vehicle_Speed_E, Average_V_C_E)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_E)
            Average_V_C.append(Average_V_C_E)
            #Node_F
            ids = self.k.vehicle.get_ids()
            Vehicle_in_F = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogF_Edges]# vehicles on different edges
            Vehicle_ID_Edge_F = flatten(clean_nested(Vehicle_in_F))
            Vehicle_Speed_F = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_F]
            Average_Vehicle_Speed_F = np.mean(Vehicle_Speed_F)
            #get lanes number and length of the edges
            V_C_F = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogF_Edges]
            V_C_F = [i for i in V_C_F if i != 0]
            Average_V_C_F = np.mean(V_C_F)
            if Average_V_C_F > 5.0 and Average_Vehicle_Speed_F == 0.0:
                Average_Vehicle_Speed_F = 8
            if Average_V_C_F > 0.0 and Average_V_C_F <= 5.0 and Average_Vehicle_Speed_F == 0.0:
                Average_Vehicle_Speed_F = 3
            #print(Average_Vehicle_Speed_F, Average_V_C_F)
            Average_Vehicle_Speed.append(Average_Vehicle_Speed_F)
            Average_V_C.append(Average_V_C_F)
            count_V_C = 0

            Average_Speed = np.array(Average_Vehicle_Speed).reshape(-1,1)
            updated_Average_Speed = np.nan_to_num(Average_Speed)
            Ratio_V_C = np.array(Average_V_C).reshape(-1,1)#reshape to one column
            updated_Ratio_V_C = np.nan_to_num(Ratio_V_C)
            for i in range(0,len(updated_Ratio_V_C)):
                if updated_Ratio_V_C[i] != 0:
                    count_V_C += 1
            #print(count_V_C)
            self.average_speed = np.sum(updated_Average_Speed)/count_V_C
            #get two vectors as a Feature matrix shape = N*F N=edge number F = features number
            states = np.c_[updated_Average_Speed,updated_Ratio_V_C]
            self.average_V_C = np.sum(updated_Ratio_V_C)/count_V_C
            adjacency = A_Matrix
            #print(states,adjacency)
            #print(states, adjacency)
            return states, adjacency

        for i in range(0,20):
            
            if self.time_counter == i*200:
                
                #print(self.step_counter)
                #print(self.k.vehicle.get_route('flow_20123.1'))
                #generate features for GAT
                #six fog nodes features: average vehicle speed and V/C
                Average_Vehicle_Speed = []
                Average_V_C = []
                #Node_A
                ids = self.k.vehicle.get_ids()
                Vehicle_in_A = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogA_Edges]# vehicles on different edges
                Vehicle_ID_Edge_A = flatten(clean_nested(Vehicle_in_A))
                Vehicle_Speed_A = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_A]
                Average_Vehicle_Speed_A = np.mean(Vehicle_Speed_A)
                
                #get lanes number and length of the edges
                V_C_A = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogA_Edges]
                V_C_A = [i for i in V_C_A if i != 0]
                Average_V_C_A = np.mean(V_C_A)
                
                if Average_V_C_A > 5.0 and Average_Vehicle_Speed_A == 0.0:
                    Average_Vehicle_Speed_A = 8.0
                if Average_V_C_A > 0.0 and Average_V_C_A <= 5.0 and Average_Vehicle_Speed_A == 0.0:
                    Average_Vehicle_Speed_A = 3.0
                #print(Average_Vehicle_Speed_A, Average_V_C_A)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_A)
                Average_V_C.append(Average_V_C_A)
                #Node_B
                ids = self.k.vehicle.get_ids()
                Vehicle_in_B = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogB_Edges]# vehicles on different edges
                Vehicle_ID_Edge_B = flatten(clean_nested(Vehicle_in_B))
                Vehicle_Speed_B = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_B]
                Average_Vehicle_Speed_B = np.mean(Vehicle_Speed_B)
                
                #get lanes number and length of the edges
                V_C_B = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogB_Edges]
                V_C_B = [i for i in V_C_B if i != 0]
                Average_V_C_B = np.mean(V_C_B)
                
                if Average_V_C_B > 5.0 and Average_Vehicle_Speed_B == 0.0:
                    Average_Vehicle_Speed_B = 8.0
                if Average_V_C_B > 0.0 and Average_V_C_B <= 5.0 and Average_Vehicle_Speed_B == 0.0:
                    Average_Vehicle_Speed_B = 3.0
                #print(Average_Vehicle_Speed_B, Average_V_C_B)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_B)
                Average_V_C.append(Average_V_C_B)
                
                #Node_C
                ids = self.k.vehicle.get_ids()
                Vehicle_in_C = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogC_Edges]# vehicles on different edges
                Vehicle_ID_Edge_C = flatten(clean_nested(Vehicle_in_C))
                Vehicle_Speed_C = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_C]
                Average_Vehicle_Speed_C = np.mean(Vehicle_Speed_C)
                
                #get lanes number and length of the edges
                V_C_C = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogC_Edges]
                V_C_C = [i for i in V_C_C if i != 0]
                Average_V_C_C = np.mean(V_C_C)
                
                if Average_V_C_C > 5.0 and Average_Vehicle_Speed_C == 0.0:
                    Average_Vehicle_Speed_C = 8.0
                if Average_V_C_C > 0.0 and Average_V_C_C <= 5.0 and Average_Vehicle_Speed_C == 0.0:
                    Average_Vehicle_Speed_C = 3.0
                #print(Average_Vehicle_Speed_C, Average_V_C_C)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_C)
                Average_V_C.append(Average_V_C_C)
                #Node_D
                ids = self.k.vehicle.get_ids()
                Vehicle_in_D = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogD_Edges]# vehicles on different edges
                Vehicle_ID_Edge_D = flatten(clean_nested(Vehicle_in_D))
                Vehicle_Speed_D = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_D]
                Average_Vehicle_Speed_D = np.mean(Vehicle_Speed_D)
                
                #get lanes number and length of the edges
                V_C_D = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogD_Edges]
                V_C_D = [i for i in V_C_D if i != 0]
                Average_V_C_D = np.mean(V_C_D)
                
                if Average_V_C_D > 5.0 and Average_Vehicle_Speed_D == 0.0:
                    Average_Vehicle_Speed_D = 8.0
                if Average_V_C_D > 0.0 and Average_V_C_D <= 5.0 and Average_Vehicle_Speed_D == 0.0:
                    Average_Vehicle_Speed_D = 3.0
                #print(Average_Vehicle_Speed_D, Average_V_C_D)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_D)
                Average_V_C.append(Average_V_C_D)
                #Node_E
                ids = self.k.vehicle.get_ids()
                Vehicle_in_E = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogE_Edges]# vehicles on different edges
                Vehicle_ID_Edge_E = flatten(clean_nested(Vehicle_in_E))
                Vehicle_Speed_E = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_E]
                Average_Vehicle_Speed_E = np.mean(Vehicle_Speed_E)
                
                #get lanes number and length of the edges
                V_C_E = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogE_Edges]
                V_C_E = [i for i in V_C_E if i != 0]
                Average_V_C_E = np.mean(V_C_E)
                
                if Average_V_C_E > 5.0 and Average_Vehicle_Speed_E == 0.0:
                    Average_Vehicle_Speed_E = 8.0
                if Average_V_C_E > 0.0 and Average_V_C_E <= 5.0 and Average_Vehicle_Speed_E == 0.0:
                    Average_Vehicle_Speed_E = 3.0
                #print(Average_Vehicle_Speed_E, Average_V_C_E)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_E)
                Average_V_C.append(Average_V_C_E)
                #Node_F
                ids = self.k.vehicle.get_ids()
                Vehicle_in_F = [self.k.vehicle.get_ids_by_edge(edge) for edge in FogF_Edges]# vehicles on different edges
                Vehicle_ID_Edge_F = flatten(clean_nested(Vehicle_in_F))
                Vehicle_Speed_F = [self.k.vehicle.get_speed(veh_id) for veh_id in Vehicle_ID_Edge_F]
                Average_Vehicle_Speed_F = np.mean(Vehicle_Speed_F)
                #get lanes number and length of the edges
                V_C_F = [len(self.k.vehicle.get_ids_by_edge(edge))*100/(self.k.network.num_lanes(edge)*self.k.network.edge_length(edge)) for edge in FogF_Edges]
                V_C_F = [i for i in V_C_F if i != 0]
                Average_V_C_F = np.mean(V_C_F)
                if Average_V_C_F > 5.0 and Average_Vehicle_Speed_F == 0.0:
                    Average_Vehicle_Speed_F = 8.0
                if Average_V_C_F > 0.0 and Average_V_C_F <= 5.0 and Average_Vehicle_Speed_F == 0.0:
                    Average_Vehicle_Speed_F = 3.0
                #print(Average_Vehicle_Speed_F, Average_V_C_F)
                Average_Vehicle_Speed.append(Average_Vehicle_Speed_F)
                Average_V_C.append(Average_V_C_F)
                count_V_C = 0
                
                Average_Speed = np.array(Average_Vehicle_Speed).reshape(-1,1)
                updated_Average_Speed = np.nan_to_num(Average_Speed)
                Ratio_V_C = np.array(Average_V_C).reshape(-1,1)#reshape to one column
                updated_Ratio_V_C = np.nan_to_num(Ratio_V_C)
                for i in range(0,len(updated_Ratio_V_C)):
                    if updated_Ratio_V_C[i] != 0:
                        count_V_C += 1
                #print(count_V_C)
                self.average_speed = np.sum(updated_Average_Speed)/count_V_C
                #get two vectors as a Feature matrix shape = N*F N=edge number F = features number
                states = np.c_[updated_Average_Speed,updated_Ratio_V_C]
                self.average_V_C = np.sum(updated_Ratio_V_C)/count_V_C
                adjacency = A_Matrix
                #print(states,adjacency)
                #print(states, adjacency)
                return states, adjacency
    
    def apply_rl_actions(self, rl_actions = None):
        for i in range(0,20):
            if self.time_counter == i*200 and self.done == False:
                if isinstance(rl_actions, np.ndarray):
                    rl_actions2 = rl_actions.copy()
                    rl_ids = ['Fog_A','Fog_B','Fog_C','Fog_D','Fog_E','Fog_F']
                    #print(rl_actions2)
                    self.Fog_A_weight = rl_actions2[0]
                    self.Fog_B_weight = rl_actions2[1]               
                    self.Fog_C_weight = rl_actions2[2]
                    self.Fog_D_weight = rl_actions2[3]
                    self.Fog_E_weight = rl_actions2[4]
                    self.Fog_F_weight = rl_actions2[5]
                return None
    
    
    def get_updated_route(self):
        
            
        for i in range(0,20):
            if self.time_counter == i*200:
                
                #update available routes -- new edge weights: is the fog node actions learned by the rl algorithm
                ids = self.k.vehicle.get_ids()
                for i in range(0,len(Edge_Weight)):
                    if len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i])) > 0:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = 1*(len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i]))+1)
                        Node_Weight[list(Node_Weight.keys())[i]] = 1*(len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i]))+1)
                    if len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i])) == 0:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = 1*(len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i]))+1)
                        Node_Weight[list(Node_Weight.keys())[i]] = 1*(len(self.k.vehicle.get_ids_by_edge(list(Edge_Weight.keys())[i]))+1)    
                    #update the weight that considers the Fog_Nodes_weight
                    if list(Edge_Weight.keys())[i] in FogA_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_A_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_A_weight
                    if list(Edge_Weight.keys())[i] in FogB_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_B_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_B_weight
                    if list(Edge_Weight.keys())[i] in FogC_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_C_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_C_weight
                    if list(Edge_Weight.keys())[i] in FogD_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_D_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_D_weight
                    if list(Edge_Weight.keys())[i] in FogE_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_E_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_E_weight
                    if list(Edge_Weight.keys())[i] in FogF_Edges:
                        Edge_Weight[list(Edge_Weight.keys())[i]] = Edge_Weight[list(Edge_Weight.keys())[i]] + 100*self.Fog_F_weight
                        Node_Weight[list(Node_Weight.keys())[i]] = Node_Weight[list(Node_Weight.keys())[i]] + 100*self.Fog_F_weight
                    
                #print(Edges_Weight)
                #Edges_Weight = [1*(len(self.k.vehicle.get_ids_by_edge(edge))+1) for edge in Edges]
                #start = time.time()
                #updated edges weight of the network
                for i in range(0,len(Node_Weight)):
                    G_network[list(Node_Weight.keys())[i][0]][list(Node_Weight.keys())[i][1]]['weight'] = Node_Weight[list(Node_Weight.keys())[i]]
                #cut1 = time.time()
                #print('1_for: ',cut1-start)
                Pair = self.Pair
                Sumo_Path = []
                for i in range(1,len(Pair)+1):
                    for path in k_shortest_paths(G_network, Pair[i][1], Pair[i][2], 3, weight = 'weight'):
                        path.insert(0,Pair[i][0])
                        edge_path = []
                        for j in range(0,len(path)-1):
                            edge_path.append(Node_to_Edge[(path[j],path[j+1])])
                        Sumo_Path.append(edge_path)
                
                #cut2 = time.time()
                #print('2_for: ',cut2-cut1)
                
                FIRST_EDGES = []
                for i in range(0,len(Sumo_Path)):
                    FIRST_EDGES.append(Sumo_Path[i][0])
                
                EDGES_DISTRIBUTION = unique(FIRST_EDGES)
                #print(len(EDGES_DISTRIBUTION))
                
                Probability_EDGES_DISTRIBUTION = []
                for i in range(0,len(EDGES_DISTRIBUTION)):
                    count = 0
                    for j in range(0,len(Sumo_Path)):
                        if Sumo_Path[j][0] == EDGES_DISTRIBUTION[i]:
                            count = count+1
                    probability = 1/count
                    Probability_EDGES_DISTRIBUTION.append(probability)
                #cut3 = time.time()
                #print('3_for: ',cut3-cut2)
                
                EDGES = []
                for i in range(0,len(EDGES_DISTRIBUTION)):
                    for j in range(0,len(Sumo_Path)):
                        if Sumo_Path[j][0] == EDGES_DISTRIBUTION[i]:
                            #print(EDGES_DISTRIBUTION[i])
                            Edg = (Sumo_Path[j],Probability_EDGES_DISTRIBUTION[i])
                            EDGES.append(Edg)
                #cut4 = time.time()
                #print('4_for: ',cut4-cut3)
                rts_update = {}
                for i in range(len(EDGES_DISTRIBUTION)):
                    group_EDGES = []
                    for j in range(len(EDGES)):
                        if EDGES[j][0][0] == EDGES_DISTRIBUTION[i]:
                            group_EDGES.append(EDGES[j])
                    rts_update[EDGES_DISTRIBUTION[i]] = group_EDGES
                #cut5 = time.time()
                #print('5_for: ',cut5-cut4)
                self.available_routes = rts_update

                #self.availeble_routes = self.get_state()
                
                #print(self.step_counter)
                veh_edge_ids = []
                for i in range(0,len(ids)):
                    if self.k.vehicle.get_edge(ids[i]).find('C') == -1 and len(self.k.vehicle.get_edge(ids[i])) != 0:
                        veh_edge_ids.append(ids[i])
                        

                #average_speed = [self.k.vehicle.get_speed(veh_id) for veh_id in veh_edge_ids]
                #self.average_speed = np.mean(average_speed)
                #if np.isnan(self.average_speed) == True:
                #    self.average_speed = 8
                #print("average_speed", self.average_speed)
        
                Distance = []
                for i in range(0,len(veh_edge_ids)):
                    current_route_d = self.k.vehicle.get_route(veh_edge_ids[i])
                    current_edge_d = self.k.vehicle.get_edge(veh_edge_ids[i])
                    current_edge_length_d = self.k.network.edge_length(current_edge_d)
                    for j in range(0,len(current_route_d)):
                        if current_route_d[j] == current_edge_d:#find the position of the current edge in the current route
                            future_route_d = current_route_d[j+1:-1]
                    distance = []
                    for m in range(0,len(future_route_d)):
                        distance.append(self.k.network.edge_length(future_route_d[m]))
                    np.sum(distance)
                    current_position_distance = current_edge_length_d - self.k.vehicle.get_position(veh_edge_ids[i])
                    total_distance = np.sum(distance) + current_position_distance
                    Distance.append(total_distance)
                average_distance_to_destination = np.mean(Distance)
                if np.isnan(average_distance_to_destination) == True:
                    self.average_distance_to_destination = 0
                #print("distance_to_destination", average_distance_to_destination)
                self.average_distance_to_destination = average_distance_to_destination
                #Pri = sorted(zip(Priority, source, target), reverse=False)
                Pri = sorted(zip(Distance,veh_edge_ids),reverse = False)
                #print(Pri)
                #Density over the network
                Density = [len(self.k.vehicle.get_ids_by_edge(edge)) for edge in Edges]
                updated_route = []
                for i in range(0,len(Pri)):
                    veh_current_edge = self.k.vehicle.get_edge(Pri[i][1])
                    #print(veh_current_edge)
                    #print("vehicleID:",Pri[i][1])
                    veh_current_route = self.k.vehicle.get_route(Pri[i][1])
                    #print(veh_current_edge, type(veh_current_edge))
                    Popularity = []
                    if i < 10:#nearest vehicle got shortest path
                        new_route = veh_current_route
                        #new_route = self.available_routes[veh_current_edge][0][0]
                        updated_route.append(new_route)
                        for edge in new_route:
                            index = Edges.index(edge)
                            Density[index] = Density[index] + 1
#                     Popularity = []
                    if i >= 10:
                        #print(veh_current_edge)
                        #print(self.available_routes[veh_current_edge])
                        #考虑destination相同？？
                        available_routes = len(self.available_routes[veh_current_edge])
                        #print(available_routes)
                        #print(available_routes)
                        for j in range(0,available_routes):
                            Entropy = []
                            Path_Density = [len(self.k.vehicle.get_ids_by_edge(edge)) for edge in self.available_routes[veh_current_edge][j]]
                            for k in range(0,len(self.available_routes[veh_current_edge][j][0])):
                                
                                edge_density = len(self.k.vehicle.get_ids_by_edge(self.available_routes[veh_current_edge][j][0][k]))
                                edge_ffspeed = self.k.network.speed_limit(self.available_routes[veh_current_edge][j][0][k])
                                edge_Length = self.k.network.edge_length(self.available_routes[veh_current_edge][j][0][k])
                                edge_lane = self.k.network.num_lanes(self.available_routes[veh_current_edge][j][0][k])
                                Weight_Factor = (average_ffspeed/edge_ffspeed)*edge_lane*(average_length/edge_Length)
                                footprint = Weight_Factor * edge_density
                                if edge_density == 0:
                                    Entropy.append(0)
                                if edge_density != 0:
                                    Entropy.append(-((footprint/np.sum(Path_Density))*np.log(footprint/np.sum(Path_Density))))
                            Popularity.append(math.exp(sum(Entropy)))
                        Pop_path = sorted(zip(Popularity,self.available_routes[veh_current_edge]), reverse = False)
                        new_route = Pop_path[0][1][0]
                        updated_route.append(new_route)
                        for edge in new_route:
                            index = Edges.index(edge)
                            Density[index] = Density[index] + 1
                #print(updated_route)
                #cut6 = time.time()
                #print('6_for: ',cut6-cut5)
                self.priority_routes = updated_route
            
    def compute_reward_no_rl_actions(self):
        if self.done == True:#if its the terminal, also return a reward value
            ids = self.k.vehicle.get_ids()
            Human_ID = []
            for i in range(0,len(ids)):
                if self.k.vehicle.get_type(ids[i]) == 'Human':
                    Human_ID.append(ids[i])
            #print(Human_ID)
            Human_speed = [self.k.vehicle.get_speed(veh_id) for veh_id in Human_ID]
            
            Human_Average_Speed = np.mean(Human_speed)
            w_average_speed = 1
            w_distance = 1
            #w_VC = 1
            speed_reward = 0
            intention_reward = 0
            ids = self.k.vehicle.get_ids()
            #print(ids)
            #reward for system speed: mean(speed/max_speed)
            #average free flow speed of the map: average_ffspeed
            #average current_speed is the vehicles' current speed:
#             average_speed = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]
            average_crspeed = self.average_speed
            speed_reward = (average_crspeed-5)*100
            #distance to the destination
            #distance = self.average_distance_to_destination
            #distance_reward = distance
            print("speed and distance reward:", speed_reward)#, distance_reward)
            #penalty need to be normalized between 0~1
            self.reward = w_average_speed * speed_reward #- (w_distance * distance_reward) #- (self.average_V_C * w_VC)
            return w_average_speed * speed_reward #- (w_distance * distance_reward) #- (self.average_V_C * w_VC)
            
        for i in range(0,20):
            if self.time_counter == i*200:
                ids = self.k.vehicle.get_ids()
                Human_ID = []
                for i in range(0,len(ids)):
                    if self.k.vehicle.get_type(ids[i]) == 'Human':
                        Human_ID.append(ids[i])
                #print(Human_ID)
                Human_speed = [self.k.vehicle.get_speed(veh_id) for veh_id in Human_ID]
                Human_Average_Speed = np.mean(Human_speed)
                #print(Veh_Type)
                w_average_speed = 1
                w_distance = 1
                #w_VC = 1
                speed_reward = 0
                intention_reward = 0
                ids = self.k.vehicle.get_ids()
                #print(ids)
                #reward for system speed: mean(speed/max_speed)
                #average free flow speed of the map: average_ffspeed
                #average current_speed is the vehicles' current speed:
                #average_speed = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]
                average_crspeed = Human_Average_Speed
                speed_reward = (average_crspeed-5)*100
                #distance to the destination
                #distance = self.average_distance_to_destination
                #distance_reward = distance
                print("speed and distance reward:", speed_reward)#, distance_reward)
                #penalty need to be normalized between 0~1
                self.reward = w_average_speed * speed_reward #- (w_distance * distance_reward) #- (self.average_V_C * w_VC)
                return w_average_speed * speed_reward #- (w_distance * distance_reward) #- (self.average_V_C * w_VC)
                
            
            
    def vehicle_current_edge(self):#Human vehicle goes to the last edge
        ids = self.k.vehicle.get_ids()
        Human_ID = []
        for i in range(0,len(ids)):
            if self.k.vehicle.get_type(ids[i]) == 'Human':
                Human_ID.append(ids[i])
        current_edges = [self.k.vehicle.get_edge(veh_id) for veh_id in Human_ID]
        type_current_edges = unique(current_edges)
        #print(type_current_edges)
        destination_difference = len(list(set(type_current_edges)-set(self.destination_edge)))
        #print(destination_difference)
        return destination_difference
    
    
        

In [14]:
#Routing Controller
class BackRouter(BaseRouter):
    #envObj = RayneEnv()
    def choose_route(self, env):
        available_routes_1 = env.back_available_routes['gneE180']
        available_routes_2 = env.back_available_routes['gneE193']
        available_routes_3 = env.back_available_routes['464516471#0']
        available_routes_4 = env.back_available_routes['gneE14']
        available_routes_5 = env.back_available_routes['-gneE65']
        available_routes_6 = env.back_available_routes['458180185']
        available_routes_7 = env.back_available_routes['gneE71']
        available_routes_8 = env.back_available_routes['541350711#0']
        #print(random.choice(available_routes_1)[0])
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE180':
            next_route = random.choice(available_routes_1)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE193':
            next_route = random.choice(available_routes_2)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == '464516471#0':
            next_route = random.choice(available_routes_3)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE14':
            next_route = random.choice(available_routes_4)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == '-gneE65':
            next_route = random.choice(available_routes_5)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == '458180185':
            next_route = random.choice(available_routes_6)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE71':
            next_route = random.choice(available_routes_7)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == '541350711#0':
            next_route = random.choice(available_routes_8)[0]
            return next_route


In [15]:
#Routing Controller
class EBkSPRouter(BaseRouter):
    #envObj = RayneEnv()
    def choose_route(self, env):
        
        available_routes_1 = env.available_routes['gneE14']
        available_routes_2 = env.available_routes['gneE61']
        available_routes_3 = env.available_routes['458166897#10']
        #print(random.choice(available_routes_1)[0])
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE14':
            next_route = random.choice(available_routes_1)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == 'gneE61':
            next_route = random.choice(available_routes_2)[0]
            return next_route
        if env.k.vehicle.get_edge(self.veh_id) == '458166897#10':
            next_route = random.choice(available_routes_3)[0]
            return next_route
#         if env.k.vehicle.get_edge(self.veh_id) == 'gneE193':
#             next_route = random.choice(available_routes_2)[0]
#         if env.k.vehicle.get_edge(self.veh_id) == '458166897#10':
#             next_route = random.choice(available_routes_2)[0]
#         print(next_route)
#         return next_route
#         for i in range(0,len(env.available_routes)):
#                 if env.available_routes[i][0]=='gneE14' or 'gneE193' or '458166897#10':
#                     available_set.append(env.available_routes[i])
#         if env.k.vehicle.get_edge(self.veh_id) == 'gneE14' or 'gneE193' or '458166897#10':
#             new_route = random.choice(available_set)
#             return new_route

            
        for i in range(1,20):
            if env.time_counter == i*200+1:
                
                #print(env.priority_routes)
                #updated_route_for_edge_vehicle = env.get_state()
                #print(env.step_counter)
                #print(env.updated_route())
                
                current_edges_weight = [1/(len(env.k.vehicle.get_ids_by_edge(edge))+1) for edge in Edges]#a list     
                current_edge = env.k.vehicle.get_edge(self.veh_id)
                current_edge_length = env.k.network.edge_length(current_edge)
                current_route = env.k.vehicle.get_route(self.veh_id)
                #print(env.get_state)
                if len(current_route) == 0:
                    return None
                if current_edge.find('C') != -1:#at junction
                    #new_route = env.available_routes[current_edge][0][0]
                    #return new_route
                    continue
                if current_edge.find('C') == -1:#at edges
                    for i in range(0,len(env.priority_routes)):
                        if current_edge == env.priority_routes[i][0]:
                            next_route = env.priority_routes[i]
                        if current_edge != env.priority_routes[i][0]:
                            continue
                        return next_route
            

In [16]:
vehicles.add(veh_id="Human",
             acceleration_controller=(IDMController, {}),
             lane_change_params = SumoLaneChangeParams("only_strategic_aggressive"),
             routing_controller = (EBkSPRouter,{}),
             color = 'green')
vehicles.add(veh_id="Back",
             acceleration_controller=(IDMController, {}),
             lane_change_params = SumoLaneChangeParams("only_strategic_aggressive"),
             routing_controller = (BackRouter,{}),
             color = 'white')


In [17]:
inflow = InFlows()

# inflow.add(veh_type="Human",
#                edge='198595461#0',
#                depart_lane='random',
#                depart_speed = 'random',
#                vehs_per_hour=1000,
#                number = 100)

# inflow.add(veh_type="Human",
#                edge='gneE29',
#                depart_lane='random',
#                depart_speed = 'random',
#                vehs_per_hour=1000,
#                number = 100)
# inflow.add(veh_type="Human",
#                edge='458166897#10',
#                depart_lane='random',
#                depart_speed = 'random',
#                vehs_per_hour=500,
#                number = 100)

inflow.add(veh_type="Back",
               edge='gneE180',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 100)
inflow.add(veh_type="Back",
               edge='gneE193',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 100)
inflow.add(veh_type="Back",
               edge='464516471#0',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 50)
inflow.add(veh_type="Back",
               edge='-gneE65',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 100)
inflow.add(veh_type="Back",
               edge='458180185',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 50)
inflow.add(veh_type="Back",
               edge='gneE71',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 30)
inflow.add(veh_type="Back",
               edge='541350711#0',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=1000,
               number = 50)
inflow.add(veh_type="Human",
               edge='gneE14',
               depart_lane='random',
               vehs_per_hour=100,
               depart_speed = 'random',
               number = 10)
inflow.add(veh_type="Human",
               edge='458166897#10',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=100,
               number = 10)
inflow.add(veh_type="Human",
               edge='gneE61',
               depart_lane='random',
               depart_speed = 'random',
               vehs_per_hour=100,
               number = 10)

In [18]:
net_params = NetParams(
    template=os.path.join(Net_Dir, "Manhattan_highway_shrink.net.xml"),
    inflows=inflow
)

In [19]:
target

['C57']

In [20]:
#from flow.core.experiment import Experiment

network = RayneNetwork('rayne',vehicles,net_params)

flow_params = dict(
    exp_tag='Manhattan_network',
    env_name=RayneEnv,
    network=network,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=new_initial_config,
)

# number of time steps
flow_params['env'].horizon = 2000


In [ ]:
from rl_experiment_gat import Experiment
#from flow.core.experiment import Experiment
exp = Experiment(flow_params)
exp.run(num_runs = 10, training = TRAINING, model = MODEL, debug = DEBUG)

5
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
X_in (InputLayer)               [(None, 6, 2)]       0                                            
__________________________________________________________________________________________________
encoder_1 (Dense)               (None, 6, 32)        96          X_in[0][0]                       
__________________________________________________________________________________________________
encoder_2 (Dense)               (None, 6, 32)        1056        encoder_1[0][0]                  
__________________________________________________________________________________________________
A_in (InputLayer)               [(None, 6, 6)]       0                                            
____________________________________________________________________________________________

/Users/raynedu/opt/anaconda3/envs/flow/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/raynedu/opt/anaconda3/envs/flow/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Interval 1 (0 steps performed)
True 0
[1 3 1 2 1 2]


In [ ]:
reward_after_warmups = [ 636.7424725420144, 188.8955687666607, 11.147607387079319, -40.60121658723959, -301.50736161195323, 70.31037660434524, 380.12987524865895, -2111.5539485460135, 222.36521495261488, -411.5964791860655, -490.9704396041125, -725.2739769570188, -444.31265640319594, 181.05586878737228, -511.44369401405453, -554.7780626672882, -362.2366457908175, -726.7095996426082, 315.19418452576235, -262.9767892248128, -612.1716631137239, -468.13337053376085, -828.7941221148455, -443.258947890309, -282.5365182690841, -207.40104660711165, -97.27516253600254, -428.7477942400034, -264.4168704235516, 566.9571777446021, -589.448241366081, 447.7948533272199, -110.31148017124588, 604.1291098926029, -392.7041659923434, -1106.3871079959094, 209.81847744718073, -324.4304672704143, 116.25789909692952, -778.7131934465975, 157.51444728436346, 224.6643331951981, -161.71155247736954, 373.2262584313384, -363.47287580860814, -94.19585932851486, 217.29423340030098, -165.07219468661458, 281.09328899512894, -769.5935473853357, -80.96163895235529, -58.03993123308112, -189.64725716605946, -438.31905625701194, -453.8546306849439, -90.21300550442896, -36.17180650364472, -663.85797204039, -443.61205877032654, 232.40808618012304, 714.4256160000336, -20.964876752776718, -474.30026025362355, -381.2563282901579, 373.06535722590627, -301.67537563483563, 307.24124484523367, 588.8656171774111, -732.6535286528738, 38.7920248628586, -608.3917093685242, 31.33273679473812, 73.91003505449765, 301.07887129824405, 100.3709742001958, 79.77878171933759, -25.636804603131452, -322.70032075831705, -283.6924873983531, -789.4894092601563, -319.6105719124388, -191.07110589819456, -167.3347835016265, 325.7084975080909, -160.66473124140413, -423.29477303052886, 71.1508057139401, -330.1598210051867, -632.1798927943424, 306.4718540713301, 384.1111258700408, -38.001837238712625, 597.4293932445767, -483.6005914606376, -314.16421066123974, -433.60314437620764, -226.32679772965696, 873.2507439443084, 143.11792415789796, 810.4548871840552, 118.97160409445098, -243.40836917767396, -390.6464789955902, -191.3523213896369, -402.32509447544703, 11.950371773234338, -494.95831885584255, 213.80028636167813, 299.66669570354856, 517.7019715838651, 99.1478659833712, -440.42791863885066, -853.7641549982127, -641.4553029211198, -282.1489761006351, -423.1598032163244, -7.431273699694067, -127.01027292554126, 206.32760654643621, 229.06007383333883, -441.24436795683454, -308.99176465885256, -825.9782087739317, -113.25006124162886, -505.56551995152563, -187.7921104408239, -414.8254597407169]
no_training_reward = [-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978,-556.5893522955978]
import matplotlib.pyplot as plt
# x axis values
x = []
y = []
for i in range(0,len(no_training_reward)):
    y.append(i)
for i in range(0,len(reward_after_warmups)):
    x.append(i)
plt.plot(x, reward_after_warmups)
plt.plot(y, no_training_reward)


In [ ]:
reward = [780.5002200376769, 1305.7051088924413, 1219.705778325697, 483.71853178699337, 351.4697196904801, 1371.190909514958, 716.9445474870704, 489.0338872212418, 761.9579419145266, 1466.672623588604, 3271.0483157378003, 1387.7333111609757, 1066.425169434209, 484.2370404266229, 2064.0941643535257, 1218.2098291186062, 1109.8660267151204, 1078.5063036496765, 1806.7686474267653, 1918.9054525866327, 587.4143908391112, 771.64665655505, 1625.8583656044575, 421.1201618629572, 1392.6159495482698, 1080.1779274701814, 1334.8739663192746, 987.9536306219381, 944.9677967793262, 1054.5857727792659, 511.8560970311703, 905.0554853667365, 265.2616032649009, 1774.3665487387766, -3608.799311138234, 813.2867259915456, 464.04939668675615, 1556.7584003487545, 493.1608151612927, 647.7056179012484, 835.0279157853048, 1425.0821405510628, 1733.915955148002, 1220.0639031527003, 2440.118071033371, 618.6431407640617, 442.8577991389957, 2955.4916762461507, 518.6074801364858, 918.9099642627891, 1045.3516255138486, 821.4360382672558, 1396.582252214428, 1327.9295081872128, 1050.2503973491953, 505.6156630851916, 362.13505643703706, 1236.3796261050272, 1532.2186328666353, 1050.1904662054644, 710.1493841570813, 934.7987886155752, 758.5135403694346, 669.3086136377117, 631.3752153180287, 529.9243427857189, 1056.815464319512, 1279.96590976632, 610.0603685702993, 792.4455921370089, 823.6554321666664, 916.2052308855615, 1130.8499064258704, 1334.9177011535137, 950.9980584931463, 544.7298852639135, 721.3843102111578, 816.5413603412906, 520.2041203228285, 488.3945467960841, 802.3039666189497, 1249.74078190844, 1010.9544943340024, 1136.134228050299, 1699.777616118654, 518.2033368049135, 975.0900524531148, 840.0049371991737, -3111.324283083105, 1491.7641323368425, 1599.8992002724972, 204.59718690388945, 984.6927191005784, 1216.650180781805, 1126.9351523051791, 564.3134208325714, 975.1810538781478, 429.3562516599552, 793.7436149626744, -135.4477668597376, 761.2814652786271, 553.103117060494, 928.0551995417917, -3969.5869245476906, 1401.0541800557085, 604.5499751265319, 677.182900272908, 844.716833897422, 971.5415729107131, 957.5309266394693, 1342.9493659424468, 1105.087282224931, -706.6609024647371, 680.1315631718206, 555.1278624247186, 471.2597411272481, 1397.3777958636315, 2466.511125730347, 1145.1346553815208, 537.7466028569048, 408.41196905341195, 1592.6212130760468, 1992.7731115333966, 746.145120269201, 1004.7388477227796, 911.2731211938165, 1142.0368520928653, 1458.6018594145685, 1281.0102130061875, 426.3509932721684, 263.7503101163044, 476.33066808723726, -152.04315883570752, -94.85753759172678, 1262.4795686616835, 1389.0766303865457, 1316.2105145705782, 744.0107091934372, 705.0632128472965, 946.2166116859937, 905.7959077273092, 62.314373890762965, 1540.9972755487724, 1247.1706362360665, 582.7627653832062, 940.6134053716781, 800.5728806112111, 761.6287727414114, 821.3073535896608, 725.3883947483338, 920.9458239589554, 1289.2338814580062, 925.8113487331339, 1023.3844415522685, 127.95062209029543, 321.61491134882056, 991.422409048085, 779.9141175354999, 573.6943621140704]
import matplotlib.pyplot as plt
# x axis values
x = []
for i in range(0,len(reward)):
    x.append(i)

plt.plot(x, reward)

In [ ]:
Q = [0.29041093587875366, 0.2853677570819855, 0.2709413766860962, 0.26963844895362854, 0.2771274447441101, 0.2912687659263611, 0.3006211817264557, 0.3098757565021515, 0.3340868651866913, 0.35291290283203125, 0.36979004740715027, 0.3841114342212677, 0.4059922695159912, 0.44593682885169983, 0.4631202220916748, 0.46888718008995056, 0.5000183582305908, 0.5313733220100403, 0.5542237758636475, 0.5714749693870544, 0.5907194018363953, 0.6096462607383728, 0.6238164901733398, 0.6463721990585327, 0.6391469836235046, 0.6537322998046875, 0.6565951704978943, 0.6696144342422485, 0.6817250847816467, 0.673006534576416, 0.6749143600463867, 0.6861472129821777, 0.7027091383934021, 0.7258146405220032, 0.7165306806564331, 0.7254243493080139, 0.7408388257026672, 0.7356100082397461, 0.7496446371078491, 0.7571116089820862, 0.7693856954574585, 0.7742561101913452, 0.7875519394874573, 0.7723351120948792, 0.7892045378684998, 0.810766339302063, 0.7914938926696777, 0.8040169477462769, 0.8122512698173523, 0.8048195242881775, 0.8185096979141235, 0.8247864246368408, 0.8260771036148071, 0.8299123644828796, 0.8327975273132324, 0.8302415013313293, 0.8327957391738892, 0.8386073112487793, 0.8285138607025146, 0.8446181416511536, 0.8357911705970764, 0.8465495705604553, 0.8347451686859131, 0.8330337405204773, 0.8400230407714844, 0.8392794132232666, 0.8400587439537048, 0.8463209271430969, 0.8377677202224731, 0.8305299878120422, 0.8173903226852417, 0.8230215907096863, 0.8076192736625671, 0.819622814655304, 0.8142217993736267, 0.8127434849739075, 0.8110419511795044, 0.8177180290222168, 0.8148866295814514, 0.8064740896224976, 0.8043085932731628, 0.7954412698745728, 0.7803062796592712, 0.7924894690513611, 0.7908546328544617, 0.7979015707969666, 0.8051081299781799, 0.786912202835083, 0.7915723323822021, 0.7940778732299805, 0.8026922941207886, 0.796779215335846, 0.8070719838142395, 0.7951207756996155, 0.7893868088722229, 0.7907881736755371, 0.7968496680259705, 0.8012579083442688, 0.7991980314254761, 0.8096919059753418, 0.7895989418029785, 0.8091855049133301, 0.7934607863426208, 0.8001697063446045, 0.7862347364425659, 0.8023757338523865, 0.798937201499939, 0.8009108901023865, 0.8002449870109558, 0.7806410789489746, 0.7947710752487183, 0.8008448481559753, 0.8028102517127991, 0.804084062576294, 0.8050112128257751, 0.8046239614486694, 0.8031885623931885, 0.7851974964141846, 0.7843776345252991, 0.7873272895812988, 0.7834362983703613, 0.787734866142273, 0.7885041236877441, 0.7913308143615723, 0.8055719137191772, 0.798544704914093, 0.798947811126709, 0.8120238780975342, 0.8093169331550598, 0.8183180689811707, 0.7920106053352356, 0.7794170379638672, 0.7796879410743713, 0.7819619178771973, 0.7932473421096802, 0.786932110786438, 0.8105846047401428, 0.8183767795562744, 0.8027786612510681, 0.8067644834518433, 0.7980948686599731, 0.7945581674575806, 0.7863127589225769, 0.7989619374275208, 0.7927445769309998, 0.7976534366607666, 0.7896670699119568, 0.7932884097099304, 0.7864547371864319, 0.7971600890159607, 0.7891348004341125, 0.7931997776031494, 0.7833079695701599, 0.7950159311294556, 0.7915019989013672, 0.793865978717804, 0.7841548919677734, 0.7815894484519958, 0.7876998782157898, 0.8026517033576965, 0.7982026934623718, 0.7993455529212952, 0.7964434623718262, 0.8084668517112732, 0.8059831857681274, 0.8039350509643555, 0.7906097769737244, 0.7827372550964355, 0.8058986067771912, 0.7787992358207703, 0.7914995551109314, 0.8147774338722229, 0.7884625196456909, 0.8067490458488464, 0.7760172486305237, 0.7900684475898743, 0.7669550776481628, 0.7695531249046326, 0.7901971936225891, 0.7762967348098755, 0.7858282923698425, 0.7865647077560425, 0.7851746678352356, 0.7981330752372742, 0.7838013172149658, 0.7999961972236633, 0.7959029078483582, 0.8006770014762878, 0.8123078346252441, 0.7998971343040466, 0.7902822494506836, 0.8045889139175415, 0.7917477488517761, 0.8061619400978088, 0.8078726530075073, 0.8085466623306274, 0.8186951279640198, 0.8091664910316467, 0.7949268817901611, 0.7762744426727295, 0.79861980676651, 0.7638683915138245, 0.7752103209495544, 0.7928442358970642, 0.7966061234474182, 0.7936262488365173, 0.8114914298057556, 0.8151617050170898, 0.8015748262405396, 0.7948817610740662, 0.7845839262008667, 0.7988465428352356, 0.7959967255592346, 0.7964984178543091, 0.7803757786750793, 0.7872201800346375, 0.7805848121643066, 0.7955784201622009, 0.7890058159828186, 0.7972738742828369, 0.7920458912849426, 0.778988242149353, 0.8075729608535767, 0.8231706619262695, 0.8273236155509949, 0.8166052103042603, 0.8139225244522095, 0.8186709880828857, 0.8107160925865173, 0.823101282119751, 0.815138041973114, 0.8186852931976318, 0.8121890425682068, 0.8440894484519958, 0.8701537251472473, 0.8981972932815552, 0.8918800950050354, 0.9130076766014099, 0.921907901763916, 0.9225466847419739, 0.9240994453430176, 0.9277482032775879, 0.9322553277015686, 0.9362807273864746, 0.9440135955810547, 0.9437903761863708, 0.9460049271583557, 0.9454624652862549, 0.9473560452461243, 0.9492499232292175, 0.9537582993507385, 0.9530487060546875, 0.9539713859558105, 0.9539152979850769, 0.9550743699073792, 0.9537804126739502, 0.9600473642349243, 0.9521561861038208, 0.9618611335754395, 0.9642106890678406, 0.9633259773254395, 0.9645516276359558, 0.9595581889152527, 0.9645333290100098, 0.9632556438446045, 0.9644657969474792, 0.9678977727890015, 0.9695885181427002, 0.9662961959838867, 0.9652865529060364, 0.9567566514015198, 0.9623088836669922, 0.9649953246116638, 0.9666728973388672, 0.9651812314987183, 0.9658815264701843, 0.9662711024284363, 0.9698237180709839, 0.9623552560806274, 0.9641997218132019, 0.9690147638320923, 0.9707873463630676, 0.970005452632904, 0.9693897366523743, 0.9690081477165222, 0.9749904274940491, 0.9718366265296936, 0.9751747250556946, 0.9747127294540405, 0.9750667214393616, 0.9768507480621338, 0.9741663932800293, 0.9736003875732422, 0.9797102808952332, 0.9753270745277405, 0.9713215827941895, 0.9746185541152954, 0.9768365025520325, 0.9700856804847717, 0.9755563139915466, 0.9749398231506348, 0.9748696684837341, 0.9774529337882996, 0.9777747988700867, 0.9747388958930969, 0.9754131436347961, 0.9794172644615173, 0.9737884402275085, 0.9793872237205505, 0.9744863510131836, 0.978120744228363, 0.9790136814117432, 0.9747689962387085, 0.9769587516784668, 0.9791699051856995, 0.9762275815010071, 0.9748188257217407, 0.9780040979385376, 0.977595329284668, 0.9778251051902771, 0.9822770953178406, 0.9760289192199707, 0.9810082316398621, 0.9775626063346863, 0.978029727935791, 0.975602388381958, 0.9810391664505005, 0.9745050072669983, 0.9820985794067383, 0.9836085438728333, 0.9789087176322937, 0.9792274832725525, 0.981997013092041, 0.9800156950950623, 0.9819033741950989, 0.981135368347168, 0.9796293377876282, 0.9781972765922546, 0.9799050688743591, 0.9764580726623535, 0.9773247838020325, 0.98587566614151, 0.9875490069389343, 0.9796249270439148, 0.9827631115913391, 0.9768656492233276, 0.9830953478813171, 0.9793762564659119, 0.981573760509491, 0.9829511642456055, 0.9838594198226929, 0.9801902770996094, 0.9829745292663574, 0.9818504452705383, 0.9797011017799377, 0.981116533279419, 0.9809631705284119, 0.9809918403625488, 0.9819125533103943, 0.9785463809967041, 0.9804121851921082, 0.9766826629638672, 0.9775606393814087, 0.9792520999908447, 0.9800155162811279, 0.982467770576477, 0.9842129945755005, 0.983483612537384, 0.9810500741004944, 0.9817468523979187, 0.9827975034713745, 0.9810694456100464, 0.9821562170982361, 0.9816474914550781, 0.9795659184455872, 0.9825831651687622, 0.9820070862770081, 0.980157196521759, 0.9855728149414062, 0.9785411953926086, 0.9809432625770569, 0.9824269413948059, 0.9835237860679626, 0.9848910570144653, 0.9852226376533508, 0.9813003540039062, 0.9825174808502197, 0.9818792343139648, 0.9786888957023621, 0.9798550009727478, 0.9836481213569641, 0.9861318469047546, 0.9838914275169373, 0.9824526309967041, 0.9884760975837708, 0.9844983220100403, 0.9855687022209167, 0.9848475456237793, 0.9825138449668884, 0.984453022480011, 0.9896026253700256, 0.9855141043663025, 0.987640380859375, 0.9864151477813721, 0.9857795834541321, 0.9852622151374817, 0.9848381876945496, 0.9866021275520325, 0.9833988547325134, 0.9849069714546204, 0.9828895330429077, 0.9798232913017273, 0.9850488901138306, 0.9828829765319824, 0.9829668998718262, 0.9809043407440186, 0.9857143759727478, 0.985209047794342, 0.9820559620857239, 0.9820447564125061, 0.9851248264312744, 0.9836486577987671, 0.9802981615066528, 0.9849650263786316, 0.9875397086143494, 0.9854500889778137, 0.9851610660552979, 0.9835371971130371, 0.9852572083473206, 0.9891455769538879, 0.983232319355011, 0.9858560562133789, 0.9875829815864563, 0.9822594523429871, 0.9852148294448853, 0.9803836345672607, 0.9836405515670776, 0.9818888306617737, 0.9814667701721191, 0.9847791790962219, 0.9862234592437744, 0.9865826368331909, 0.9868902564048767, 0.9856612086296082, 0.988996684551239, 0.9844478368759155, 0.9858934283256531, 0.9828675985336304, 0.9883778691291809, 0.9886638522148132, 0.9857254028320312, 0.9877610802650452, 0.9890633225440979, 0.9872762560844421, 0.9836938381195068, 0.9886641502380371, 0.9866806268692017, 0.9880675673484802, 0.9826964735984802, 0.9855396151542664, 0.9871614575386047, 0.9883261919021606, 0.9876208305358887, 0.9874815344810486, 0.985974907875061, 0.988209068775177, 0.9857537150382996, 0.984184205532074, 0.9867229461669922, 0.9917510747909546, 0.9890538454055786, 0.9900416731834412, 0.9899187684059143, 0.9898624420166016, 0.9908782839775085, 0.9887340068817139, 0.9880174994468689, 0.9922471046447754, 0.9853864908218384, 0.9907152056694031, 0.9888067841529846, 0.9899162650108337, 0.9882380366325378, 0.9908459782600403, 0.9894394278526306, 0.9874711036682129, 0.9927021861076355, 0.9912664294242859, 0.9894765019416809, 0.9919382929801941, 0.9922199845314026, 0.9932783246040344, 0.9910585284233093, 0.991331160068512, 0.9878000020980835, 0.9911757111549377, 0.9904543161392212, 0.9890645146369934, 0.99005526304245, 0.9923157691955566, 0.9898509383201599, 0.9939549565315247, 0.990310788154602, 0.9887101054191589, 0.9930949211120605, 0.9882848858833313, 0.9908525347709656, 0.9916731119155884, 0.9925718307495117, 0.9922336339950562, 0.9901838898658752, 0.9919151663780212, 0.9911267161369324, 0.9938932061195374, 0.9896647930145264, 0.9918063282966614, 0.9934234619140625, 0.9910873174667358, 0.9891045093536377, 0.9903824925422668, 0.9975175857543945, 0.990504264831543, 0.9901130795478821, 0.989979088306427, 0.9883546829223633, 0.9942041635513306, 0.991461455821991, 0.9925982356071472, 0.9905880689620972, 0.9906293153762817, 0.9925632476806641, 0.9919647574424744, 0.9919837713241577, 0.9914835095405579, 0.9938540458679199, 0.9937419295310974, 0.9890226721763611, 0.9928820133209229, 0.9908859729766846, 0.9924613237380981, 0.9931383728981018, 0.9874057173728943, 0.9895474314689636, 0.9921446442604065, 0.9908812046051025, 0.9909883737564087, 0.9929434061050415, 0.9896602630615234, 0.990938663482666, 0.9919626712799072, 0.9955852031707764, 0.9926338791847229, 0.9925386309623718, 0.9941660165786743, 0.9908117651939392, 0.9919759631156921, 0.9916664361953735, 0.9923821091651917, 0.9951064586639404, 0.9934247136116028, 0.9890087246894836, 0.9936960935592651, 0.9926043748855591, 0.9923499226570129, 0.9932560920715332, 0.993694007396698, 0.9913198351860046, 0.9942753314971924, 0.9922311902046204, 0.9927870631217957, 0.9935935139656067, 0.9935097694396973, 0.9951972365379333, 0.9952638149261475, 0.9924976229667664, 0.9913418889045715, 0.9919655919075012, 0.9937714338302612, 0.9944983124732971, 0.9944444894790649, 0.9938889145851135, 0.9921876192092896, 0.9948753714561462, 0.9933233261108398, 0.9913415312767029, 0.9923257231712341, 0.9925858378410339, 0.9920368790626526, 0.9953705072402954, 0.9934787154197693, 0.9922394156455994, 0.9951731562614441, 0.993920624256134, 0.9924801588058472, 0.9922208189964294, 0.9953020215034485, 0.9912531971931458, 0.9954147934913635, 0.9941109418869019, 0.9948261976242065, 0.9948663115501404, 0.9918748140335083, 0.9928853511810303, 0.9935488700866699, 0.9924823045730591, 0.9964309334754944, 0.993004322052002, 0.9917173385620117, 0.9900192022323608, 0.9933185577392578, 0.9913316369056702, 0.9901863932609558, 0.9953145384788513, 0.9914490580558777, 0.993312418460846, 0.9926300048828125, 0.9935221672058105, 0.9938256144523621, 0.9942533373832703, 0.9916747808456421, 0.9955092072486877, 0.992498517036438, 0.9923310279846191, 0.9946588277816772, 0.9945971965789795, 0.9944078922271729, 0.9930809736251831, 0.994323194026947, 0.9918245077133179, 0.9920346140861511, 0.9938082695007324, 0.9945815205574036, 0.9932928085327148, 0.9938471913337708, 0.9936124682426453, 0.9949092864990234, 0.9938758611679077, 0.9928030967712402, 0.9946848154067993, 0.9934867024421692, 0.9930370450019836, 0.9930291175842285, 0.9949564337730408, 0.9945288300514221, 0.9956464767456055, 0.9940241575241089, 0.9945768117904663, 0.9936311841011047, 0.9940866231918335, 0.9948039650917053, 0.9952012896537781, 0.9956225752830505, 0.9925124049186707, 0.9952343106269836, 0.9919994473457336, 0.9942726492881775, 0.9956003427505493, 0.9947884678840637, 0.9955776929855347, 0.99118971824646, 0.9936206936836243, 0.9952507019042969, 0.9918840527534485, 0.9952556490898132, 0.9945534467697144, 0.9948355555534363, 0.9933909773826599, 0.9918293952941895, 0.993932843208313, 0.9950146675109863, 0.9944319128990173, 0.993262767791748, 0.9950270056724548, 0.9929476380348206, 0.9932682514190674, 0.9937395453453064, 0.9940792918205261, 0.9960930943489075, 0.9925793409347534, 0.9933218359947205, 0.9930593967437744, 0.9944566488265991, 0.9946630597114563, 0.9954872131347656, 0.9945619106292725, 0.9945927858352661, 0.9933871626853943, 0.9946737885475159, 0.9938978552818298, 0.9950600266456604, 0.993017852306366, 0.9933239817619324, 0.9934324026107788, 0.9949625730514526, 0.9931133389472961, 0.9971817135810852, 0.9946532845497131, 0.9964650869369507, 0.9934196472167969, 0.990967869758606, 0.9959577322006226, 0.995291531085968, 0.9947443008422852, 0.9921868443489075, 0.9966574907302856, 0.9947754740715027, 0.995558500289917, 0.993920624256134, 0.9962975978851318, 0.9966144561767578, 0.9959555864334106, 0.9941864013671875, 0.995068371295929, 0.9961753487586975, 0.9938047528266907, 0.9963696599006653, 0.9967041015625, 0.9951242208480835, 0.9919202327728271, 0.9965178966522217, 0.9928281903266907, 0.994839072227478, 0.9934235215187073, 0.9944016337394714, 0.9938422441482544, 0.9951645731925964, 0.9964156746864319, 0.9942939877510071, 0.9940498471260071, 0.9948113560676575, 0.9940966367721558, 0.9915880560874939, 0.9944100379943848, 0.9948477745056152, 0.9935778975486755, 0.9941889047622681, 0.993536651134491, 0.993257999420166, 0.9956264495849609, 0.9959701299667358, 0.9952016472816467, 0.9957294464111328, 0.9972791075706482, 0.9982781410217285, 0.9969302415847778, 0.9975230693817139, 0.9947876930236816, 0.9921277761459351, 0.9935563206672668, 0.995928943157196, 0.9961382746696472, 0.9956985712051392, 0.9952583909034729, 0.9966252446174622, 0.9961233139038086, 0.9954665899276733, 0.9931014776229858, 0.9948180317878723, 0.9926602244377136, 0.9973406195640564, 0.9951266050338745, 0.9953396320343018, 0.994351863861084, 0.9956626892089844, 0.9963170289993286, 0.9955081939697266, 0.9955921173095703, 0.9944888949394226, 0.9957934617996216, 0.99371737241745, 0.9956411719322205, 0.9957320094108582, 0.993346095085144, 0.9956523776054382, 0.9919404983520508, 0.9961731433868408, 0.9931103587150574, 0.9955372214317322, 0.9951614737510681, 0.9962298274040222, 0.9936598539352417, 0.9949310421943665, 0.9961848258972168, 0.9954789280891418, 0.9962073564529419, 0.9971475601196289, 0.9962713122367859, 0.9953091144561768, 0.9978014230728149, 0.9947420954704285, 0.9963247179985046, 0.9947313070297241, 0.9981593489646912, 0.9970941543579102, 0.9949893951416016, 0.9929768443107605, 0.9927968382835388, 0.9948888421058655, 0.994975745677948, 0.9934794306755066, 0.9967590570449829, 0.9944159984588623, 0.9952309727668762, 0.9956110119819641, 0.9948104023933411, 0.9971665143966675, 0.9935162663459778, 0.9956894516944885, 0.9950453042984009, 0.9961147308349609, 0.99517822265625, 0.9967390894889832, 0.9965364336967468, 0.9949166774749756, 0.9971661567687988, 0.9945768713951111, 0.9940139651298523, 0.993543803691864, 0.9978311657905579, 0.9960773587226868, 0.9969552159309387, 0.995572566986084, 0.996491551399231, 0.9950330853462219, 0.9956178665161133, 0.9955888390541077, 0.9936329126358032, 0.9942917227745056, 0.9968496561050415, 0.9962091445922852, 0.9966399073600769, 0.9954026341438293, 0.9967782497406006, 0.9947487115859985, 0.9958095550537109, 0.9958826899528503, 0.9963420629501343, 0.9931877851486206, 0.996897280216217, 0.9953402280807495, 0.9960978627204895, 0.9964504241943359, 0.9977405071258545, 0.9989274144172668, 0.9963825345039368, 0.9965537190437317, 0.9963199496269226, 0.9946860671043396, 0.9969895482063293, 0.9980340003967285, 0.9930203557014465, 0.9958435893058777, 0.9929027557373047, 0.9922552704811096, 0.9960238933563232, 0.9957119822502136, 0.9996816515922546, 0.9991636276245117, 0.995154857635498, 0.9960902333259583, 0.993591845035553, 0.998585045337677, 0.9982293248176575, 0.993277370929718, 0.9969584345817566, 0.9961633682250977, 0.9923135638237, 0.9979469776153564, 0.9974830150604248, 0.9984967708587646, 0.997572124004364, 0.9918280243873596, 0.9964703917503357, 0.9955699443817139, 0.994583785533905, 0.9898269176483154, 0.9925899505615234, 0.9933788776397705, 0.9909707903862, 0.9935646057128906, 0.9986842274665833, 0.999607264995575, 0.9953837394714355, 0.9975350499153137, 0.999675989151001, 0.9972769618034363, 0.9925136566162109, 0.9982107281684875, 0.9974520206451416, 0.9923661351203918, 0.9917104840278625, 0.9910451769828796, 0.9945485591888428, 0.9928110241889954, 0.9956388473510742, 0.9950169920921326, 0.9929916858673096, 0.9984521865844727, 0.9985834956169128, 0.9996796250343323, 0.9948561787605286, 0.9971246123313904, 0.9957866668701172, 0.9944543838500977, 0.9966199994087219, 0.9965619444847107, 0.9994635581970215, 0.9969161152839661, 0.9987234473228455, 0.9963914752006531, 0.9955883622169495, 0.9973358511924744, 0.9992104172706604, 0.9967657923698425, 0.9936931729316711, 0.9956620335578918, 0.9947386384010315, 0.9983277320861816, 0.9900910258293152, 0.9952325820922852, 0.9937775731086731, 0.9947095513343811, 0.9926116466522217, 0.9928107261657715, 0.989797055721283, 0.9971966743469238, 0.9942765235900879, 0.995232343673706, 0.996814489364624, 0.9980248808860779, 0.9953822493553162, 0.9998824000358582, 0.9930469393730164, 0.9914531707763672, 0.9944879412651062, 0.9953498244285583, 0.9976625442504883, 0.9944536089897156, 0.9954667091369629, 0.9974896907806396, 0.998512327671051, 0.995751678943634, 0.9986283779144287, 0.9944304823875427, 0.9971916675567627, 0.9967599511146545, 0.9968449473381042]
import matplotlib.pyplot as plt
# x axis values
x = []
for i in range(0,len(Q)):
    x.append(i)

plt.plot(x, Q)

In [ ]:
# Return = [-492.6448665011533,
#   -1221.0740104995211,
#   -943.6261482196085,
#   -58.09843107347052,
#   -78.62367694256915,
#   -1440.696827695721,
#   131.94022206048209,
#   -976.8039092846325,
#   -528.6731252687971,
#   42.407250469011785]
Return = [297922.953125, 292260.71875, 274333.53125, 306686.875, 331138.78125, 269791.6875, 277742.1875, 298851.28125, 297279.28125, 374490.40625, 260026.703125, 330984.3125, 317068.28125, 320785.78125, 307909.03125, 340211.4375, 289344.5625, 347606.5, 257363.4375, 243808.125, 344927.15625, 304587.6875, 281712.21875, 295003.09375, 273276.5, 292357.21875, 281061.8125, 352160.3125, 284759.5, 274498.90625, 318436.65625, 350125.90625, 326870.9375, 384223.59375, 306388.96875, 306379.40625, 314746.28125, 258285.75, 318067.625, 285488.25, 325535.0625, 337418.71875, 346385.46875, 282049.1875, 290456.40625, 229720.671875, 235825.453125, 298575.46875, 345145.21875, 336931.1875, 324738.09375, 356009.625, 335813.21875, 312143.03125, 370082.5, 322473.625, 315513.21875, 272661.5, 308359.71875, 285305.9375, 352375.53125, 302129.03125, 320310.84375, 213069.390625, 290679.46875, 254283.25, 388863.84375, 233255.359375, 293499.65625, 326068.71875, 272406.96875, 351442.5, 335868.21875, 287986.21875, 289342.65625, 303968.0, 293402.84375, 273294.09375, 305400.40625, 230042.921875, 420572.375, 339168.59375, 335529.84375, 270630.15625, 353443.25, 270868.25, 308699.28125, 274120.40625, 345008.84375, 318066.90625, 319100.25, 353462.0, 287510.875, 301219.78125, 309884.625, 270505.25, 317582.28125, 265226.375, 273693.46875, 281460.75, 279512.53125, 313799.59375, 254827.546875, 249248.625, 351202.78125, 267875.59375, 257526.890625, 269778.90625, 295966.78125, 289190.40625, 306218.65625, 360740.5, 309606.21875, 264005.96875, 294059.0625, 268084.96875, 299194.46875, 239990.25, 269493.65625, 296373.59375, 392566.34375, 377126.40625, 223321.53125, 242007.640625, 291695.59375, 252889.640625, 312263.875, 242005.5625, 366401.0, 307643.65625, 275921.46875]

#mean_Return = np.mean(Return)
#print(mean_Return)
import matplotlib.pyplot as plt
# x axis values
x = []
for i in range(0,len(Return)):
    x.append(i)

plt.plot(x, Return)

In [ ]:
loss = [2.158008337020874, 1.8901901245117188, 1.9107091426849365, 2.320629835128784, 1.3875653743743896, 1.5423262119293213, 1.8199167251586914, 1.73820161819458, 1.6956167221069336, 2.1105868816375732, 1.8036872148513794, 1.7381200790405273, 1.8577073812484741, 1.530676007270813, 1.9772332906723022, 2.3255393505096436, 2.487169027328491, 2.884625196456909, 2.406586170196533, 1.8890223503112793, 2.344531297683716, 1.6491860151290894, 1.791161298751831, 1.586785912513733, 1.9139690399169922, 1.933322548866272, 1.5305131673812866, 1.568853497505188, 1.74168860912323, 1.4650115966796875, 1.8951538801193237, 1.995972752571106, 1.5818653106689453, 1.8386621475219727, 1.5407207012176514, 1.6713825464248657, 1.6026906967163086, 1.4868236780166626, 1.2955297231674194, 1.2486509084701538, 1.9331514835357666, 1.6674973964691162, 1.3057307004928589, 1.695824384689331, 1.5440795421600342, 1.239121913909912, 1.366703748703003, 1.423190951347351, 1.266065001487732, 1.1412086486816406, 1.3912473917007446, 1.513068437576294, 1.312447190284729, 1.2994885444641113, 1.0586655139923096, 1.1801589727401733, 1.147250771522522, 1.2202694416046143, 1.3316255807876587, 1.2465823888778687, 1.3397150039672852, 1.441038727760315, 1.0651893615722656, 1.2024734020233154, 1.1122510433197021, 1.031317114830017, 1.6558566093444824, 1.169603705406189, 1.062775731086731, 1.0690773725509644, 0.8963125348091125, 1.2736650705337524, 1.0045671463012695, 1.1268216371536255, 1.177176594734192, 1.1445420980453491, 1.1398905515670776, 0.9181227684020996, 1.32129967212677, 1.2925913333892822, 1.0460273027420044, 1.0536502599716187, 1.126976728439331, 1.1921330690383911, 1.0317496061325073, 1.1370476484298706, 0.9741488695144653, 1.1587141752243042, 0.7774167060852051, 1.035581350326538, 1.1212260723114014, 0.9542373418807983, 1.0453628301620483, 1.101041555404663, 1.110979676246643, 1.210304856300354, 1.1277365684509277, 1.100043773651123, 1.289734125137329, 1.3596962690353394, 1.169083833694458, 1.452675700187683, 1.1904070377349854, 0.9264156222343445, 0.9411785006523132, 1.0199881792068481, 1.1674542427062988, 1.102004885673523, 0.8813647031784058, 1.2339282035827637, 1.049241542816162, 1.3011826276779175, 1.2009339332580566, 1.1414542198181152, 0.8151045441627502, 1.178253173828125, 1.0148972272872925, 0.8981623649597168, 1.108582854270935, 1.1228889226913452, 1.3427683115005493, 1.1991382837295532, 1.1624982357025146, 0.9108545184135437, 1.0060079097747803, 1.127352237701416, 1.0594639778137207, 0.9889044165611267, 1.0802607536315918, 1.0947742462158203, 1.03163480758667, 1.128275752067566, 1.0169569253921509, 1.2463915348052979, 1.1246533393859863, 0.9690927267074585, 1.176383137702942, 0.9656721949577332, 1.2162402868270874, 1.2309707403182983, 0.8971561193466187, 1.2523221969604492, 1.084394097328186, 1.0284205675125122, 0.8626782298088074, 0.8764522075653076, 1.0034430027008057, 1.1887296438217163, 0.9107494354248047, 1.1211012601852417, 0.9298595786094666, 0.8184435367584229, 1.0221818685531616, 0.9673759341239929, 1.194320797920227, 0.8226406574249268, 0.9940364360809326, 1.0981473922729492, 1.0604437589645386, 1.0187292098999023, 1.001577377319336, 0.8382709622383118, 0.896325409412384, 1.0295625925064087, 1.094841718673706, 0.8166203498840332, 1.0982314348220825, 1.058896541595459, 1.0788789987564087, 1.102892279624939, 0.9749394655227661, 1.1381522417068481, 1.0816216468811035, 0.9495993852615356, 0.7904806137084961, 0.9764676094055176, 0.8902044892311096, 0.8904398083686829, 0.947108805179596, 0.9656370878219604, 0.9162982106208801, 0.9250541925430298, 1.002881646156311, 0.9176028370857239, 1.1208531856536865, 1.1031544208526611, 1.108546495437622, 0.9760572910308838, 0.9739154577255249, 0.7838327288627625, 1.0203769207000732, 1.0107245445251465, 1.0206506252288818, 0.9456674456596375, 0.9036224484443665, 0.8796261548995972, 0.8911008834838867, 0.990048885345459, 1.0684494972229004, 0.9854139685630798, 0.8539144992828369, 1.05721116065979, 1.1549279689788818, 0.9837448000907898, 0.9278686046600342, 0.6826017498970032, 1.1049631834030151, 0.8410326242446899, 0.9282733798027039, 0.9957634806632996, 1.1793075799942017, 0.9673442244529724, 0.7505270838737488, 1.1422852277755737, 0.9325475692749023, 1.0979365110397339, 1.002498984336853, 0.8569941520690918, 0.8573206663131714, 0.9914385676383972, 0.6853908896446228, 0.9397894740104675, 0.9842063188552856, 1.0400890111923218, 1.060488224029541, 1.0586645603179932, 1.0459648370742798, 1.0416693687438965, 0.8773372769355774, 1.0475692749023438, 1.082247257232666, 1.3765901327133179, 1.4036930799484253, 1.6399768590927124, 1.3360222578048706, 1.448853611946106, 1.6747134923934937, 1.3780006170272827, 1.5485949516296387, 1.8249468803405762, 1.0996043682098389, 1.5420114994049072, 1.2704896926879883, 1.7219409942626953, 1.5853863954544067, 1.2857745885849, 1.1789683103561401, 1.603165626525879, 1.470194697380066, 1.5179507732391357, 1.3340638875961304, 1.318203091621399, 1.2187832593917847, 1.009989857673645, 1.0976753234863281, 0.9725122451782227, 1.4590586423873901, 1.3099602460861206, 1.30669105052948, 1.1884584426879883, 1.3617604970932007, 1.3682861328125, 1.312318205833435, 1.362947702407837, 1.615107536315918, 1.2399966716766357, 1.5998506546020508, 1.1419507265090942, 1.299250602722168, 1.4894737005233765, 1.1594765186309814, 1.2013393640518188, 1.2169073820114136, 1.3470895290374756, 1.3897321224212646, 1.2772539854049683, 1.673892855644226, 1.4844590425491333, 1.3665072917938232, 1.806644082069397, 1.1076581478118896, 1.4513750076293945, 1.7626745700836182, 1.4373477697372437, 1.3135870695114136, 1.167661190032959, 1.2594000101089478, 1.2886666059494019, 1.6537094116210938, 1.1436853408813477, 1.326123833656311, 1.0892844200134277, 1.3382771015167236, 0.9798142910003662, 1.8965263366699219, 1.4593877792358398, 1.2444872856140137, 1.2127819061279297, 1.1908657550811768, 1.3431541919708252, 1.419561743736267, 1.2780828475952148, 1.3304113149642944, 0.9113187789916992, 1.3986276388168335, 1.2139215469360352, 1.1245073080062866, 1.6208068132400513, 1.1480618715286255, 1.0219498872756958, 1.0915249586105347, 1.4159988164901733, 1.3180835247039795, 1.3082420825958252, 1.1375833749771118, 1.5640455484390259, 1.632983922958374, 1.6873064041137695, 1.0826737880706787, 1.586731195449829, 1.5268054008483887, 1.417111873626709, 1.127793550491333, 1.210960030555725, 1.6714695692062378, 1.3392016887664795, 1.467118501663208, 1.2634028196334839, 1.2752031087875366, 1.438744068145752, 1.326097011566162, 1.3580533266067505, 1.468598484992981, 0.9042900204658508, 0.9594721794128418, 1.5540580749511719, 1.708518147468567, 1.338351845741272, 1.5608787536621094, 1.6678465604782104, 1.2696435451507568, 1.3555521965026855, 1.1668548583984375, 1.5170987844467163, 1.2531629800796509, 1.7217435836791992, 1.7365703582763672, 0.8513355255126953, 1.0277460813522339, 1.32114839553833, 1.624659776687622, 1.2298622131347656, 1.4163504838943481, 1.226549506187439, 1.2264988422393799, 1.3175607919692993, 1.3255805969238281, 1.2787604331970215, 1.4810796976089478, 0.9377219080924988, 1.4982283115386963, 1.2630791664123535, 1.2202742099761963, 1.199852466583252, 1.2746931314468384, 1.650911569595337, 1.624584436416626, 1.4615845680236816, 1.2992489337921143, 1.911542296409607, 1.5873866081237793, 1.497377634048462, 1.1691454648971558, 1.2269554138183594, 1.0984388589859009, 1.0379441976547241, 1.3787733316421509, 1.486156940460205, 1.6169500350952148, 1.4368008375167847, 1.138413429260254, 1.5362426042556763, 1.40812349319458, 1.4047720432281494, 1.4231810569763184, 1.4379881620407104, 1.54629385471344, 1.5785446166992188, 1.1863467693328857, 1.4872690439224243, 1.460243821144104, 1.2306562662124634, 1.713366150856018, 1.3637548685073853, 1.550014853477478, 1.30691659450531, 1.4010870456695557, 1.2304199934005737, 1.124393105506897, 1.3749338388442993, 1.439649224281311, 0.8211910128593445, 1.4022928476333618, 1.4129575490951538, 1.4516640901565552, 1.2247209548950195, 0.9354950189590454, 1.624399185180664, 1.1498491764068604, 1.1593936681747437, 1.6619186401367188, 1.28665292263031, 1.3560512065887451, 1.4628196954727173, 1.2136205434799194, 1.0772836208343506, 1.63313627243042, 1.2146592140197754, 1.2649370431900024, 1.7678229808807373, 1.6720002889633179, 1.4568507671356201, 1.0599039793014526, 0.9362236857414246, 1.5972703695297241, 1.0461196899414062, 1.5440075397491455, 1.3949209451675415, 1.3231911659240723, 1.3543676137924194, 1.5052448511123657, 1.5705790519714355, 1.4043196439743042, 1.5038965940475464, 1.00846266746521, 1.2978227138519287, 1.3342921733856201, 1.0992778539657593, 1.6390806436538696, 1.4513466358184814, 1.2339612245559692, 1.5947115421295166, 1.1092599630355835, 1.690615177154541, 1.6464555263519287, 1.2934517860412598, 1.152023434638977, 1.437238097190857, 1.5637884140014648, 1.6058027744293213, 1.2917536497116089, 1.52046537399292, 1.2435356378555298, 1.0686511993408203, 1.0993980169296265, 1.2349439859390259, 1.6743299961090088, 1.4466724395751953, 1.5581308603286743, 1.4565304517745972, 1.4339818954467773, 1.380636215209961, 1.2311418056488037, 1.3459765911102295, 1.4787094593048096, 1.3661187887191772, 1.7174853086471558, 1.5413540601730347, 1.5289719104766846, 1.3560423851013184, 1.6668288707733154, 1.7579845190048218, 1.9009981155395508, 1.8062306642532349, 1.4346963167190552, 1.8267561197280884, 1.4271210432052612, 1.8833280801773071, 1.7796199321746826, 1.6002942323684692, 1.5741286277770996, 1.75558340549469, 1.7131253480911255, 1.3993409872055054, 1.7396012544631958, 1.5862984657287598, 1.903863549232483, 1.4183496236801147, 1.3928228616714478, 1.6667168140411377, 2.0656158924102783, 1.5364058017730713, 1.5318917036056519, 1.1478493213653564, 1.5715745687484741, 1.2988768815994263, 1.5616538524627686, 1.228143334388733, 1.5048713684082031, 1.500063419342041, 1.3016510009765625, 1.4043347835540771, 1.7209926843643188, 1.5515034198760986, 1.508127212524414, 1.3282681703567505, 1.511260986328125, 1.4499839544296265, 1.6463165283203125, 1.4108139276504517, 1.6785695552825928, 1.9935331344604492, 1.2056397199630737, 1.6194963455200195, 1.799887776374817, 1.5972464084625244, 2.391903877258301, 1.8198192119598389, 1.3535175323486328, 1.1641862392425537, 1.1993688344955444, 1.6413108110427856, 1.5109256505966187, 1.418256163597107, 1.518754005432129, 1.6474523544311523, 1.658118724822998, 1.6878373622894287, 1.885144829750061, 1.9477602243423462, 1.6098260879516602, 1.452104091644287, 1.6509336233139038, 1.4880090951919556, 1.2887157201766968, 1.8214068412780762, 1.5713088512420654, 1.5551172494888306, 1.4650516510009766, 1.5981976985931396, 2.6123855113983154, 1.5660425424575806, 1.6920439004898071, 1.6591798067092896, 1.524519920349121, 1.2988567352294922, 1.468775987625122, 1.403914213180542, 1.5903054475784302, 1.8011835813522339, 1.6352663040161133, 1.1626946926116943, 1.8120183944702148, 1.478791356086731, 1.4140938520431519, 1.5101721286773682, 1.466855525970459, 1.4644864797592163, 1.6008063554763794, 1.374530553817749, 1.6918575763702393, 1.5655033588409424, 1.7366548776626587, 1.225831151008606, 1.5894020795822144, 1.7016717195510864, 1.4729686975479126, 1.4806650876998901, 1.4953551292419434, 1.6037474870681763, 1.3147718906402588, 1.5802268981933594, 1.6570626497268677, 1.6294612884521484, 1.5170478820800781, 1.5253393650054932, 1.5848300457000732, 2.078503131866455, 1.6038818359375, 1.4609744548797607, 1.064919352531433, 1.5034410953521729, 1.2438615560531616, 1.6973267793655396, 1.3291579484939575, 1.250352144241333, 1.127256989479065, 1.6421438455581665, 1.3079805374145508, 1.3010375499725342, 1.9194825887680054, 1.4522790908813477, 1.8680349588394165, 1.644383430480957, 1.699589729309082, 1.5247944593429565, 1.6519545316696167, 1.3083040714263916, 1.41563880443573, 1.094937801361084, 1.7666280269622803, 1.6504878997802734, 1.5186680555343628, 1.7893757820129395, 1.530551552772522, 0.8325673937797546, 1.250707983970642, 1.3152635097503662, 1.5648558139801025, 1.514253854751587, 1.6951286792755127, 1.468086838722229, 1.5519790649414062, 1.560236930847168, 1.6612962484359741, 1.5261142253875732, 1.0825929641723633, 1.529143214225769, 1.098484992980957, 1.4172461032867432, 1.5218082666397095, 1.4970550537109375, 1.3949878215789795, 1.2258710861206055, 1.462105631828308, 1.5890083312988281, 1.4226194620132446, 1.3405460119247437, 1.6619874238967896, 1.6074941158294678, 1.5439387559890747, 1.4168554544448853, 1.170778512954712, 1.2919704914093018, 1.6572816371917725, 1.2101662158966064, 1.9753202199935913, 1.5594205856323242, 1.3830314874649048, 1.7220656871795654, 1.0577775239944458, 1.5499396324157715, 1.5562278032302856, 1.4788055419921875, 1.2647253274917603, 1.337265968322754, 1.5487254858016968, 1.8521827459335327, 1.610135555267334, 1.3549338579177856, 1.23055899143219, 1.4972070455551147, 1.4549164772033691, 1.3925471305847168, 1.524778127670288, 1.4993085861206055, 1.4260847568511963, 1.740402340888977, 1.5759936571121216, 0.892176628112793, 1.1457792520523071, 1.3281803131103516, 0.9697213172912598, 1.6694759130477905, 1.5608398914337158, 1.4376839399337769, 2.0165159702301025, 1.8017897605895996, 1.1798875331878662, 1.4976123571395874, 1.533894419670105, 1.5344181060791016, 1.3803783655166626, 1.1737440824508667, 1.564523458480835, 1.2575780153274536, 1.5176843404769897, 1.3185378313064575, 1.6135549545288086, 1.836023211479187, 1.4755287170410156, 1.3430540561676025, 1.4493781328201294, 1.7130968570709229, 1.8700122833251953, 1.470604658126831, 1.944464087486267, 1.367382526397705, 1.739599585533142, 1.6712195873260498, 1.3292231559753418, 1.8738466501235962, 1.131459355354309, 1.480669617652893, 1.7416810989379883, 1.774764060974121, 1.4534095525741577, 1.3735202550888062, 1.2988032102584839, 1.2642425298690796, 1.354645848274231, 1.7072046995162964, 1.7689316272735596, 1.4632798433303833, 1.5406646728515625, 1.3226169347763062, 2.002894401550293, 1.6127004623413086, 1.5883431434631348, 1.3938888311386108, 1.5448057651519775, 1.5409313440322876, 1.7326444387435913, 1.6417226791381836, 1.132077932357788, 1.676547646522522, 1.7801460027694702, 1.0630152225494385, 1.6225289106369019, 1.7028658390045166, 1.5098243951797485, 1.3986327648162842, 1.583795428276062, 1.5518600940704346, 1.5424569845199585, 1.7801318168640137, 1.3185276985168457, 1.3706351518630981, 1.106574535369873, 1.536299467086792, 1.4960213899612427, 1.2632055282592773, 1.2910538911819458, 1.3772071599960327, 1.5407013893127441, 1.8894498348236084, 1.386296033859253, 1.6175755262374878, 1.361806035041809, 1.4107459783554077, 1.3150875568389893, 1.7825511693954468, 1.5176515579223633, 1.2264360189437866, 0.9810830950737, 1.5186207294464111, 2.0671582221984863, 1.366609811782837, 1.253494143486023, 1.3744949102401733, 1.548675537109375, 1.3019064664840698, 1.6460716724395752, 1.3582268953323364, 1.3546720743179321, 1.2076772451400757, 1.6756120920181274, 1.3860385417938232, 1.2654695510864258, 1.2763577699661255, 1.8634991645812988, 1.2953652143478394, 1.3333650827407837, 1.543807864189148, 1.6474262475967407, 1.2485501766204834, 1.6115491390228271, 1.6952857971191406, 1.4715462923049927, 1.318655014038086, 1.575294017791748, 1.2814445495605469, 1.3529294729232788, 1.4615744352340698, 1.5862737894058228, 1.4556770324707031, 2.0725791454315186, 1.4132026433944702, 1.8579190969467163, 1.3716503381729126, 1.7929675579071045, 0.9870813488960266, 1.3038233518600464, 1.7164692878723145, 1.7015928030014038, 1.8065332174301147, 1.298064947128296, 1.4112170934677124, 1.894418478012085, 1.3713301420211792, 1.4874664545059204, 1.4031426906585693, 1.3431423902511597, 1.1506094932556152, 1.4455443620681763, 1.0254753828048706, 1.8378777503967285, 1.358366847038269, 1.7156902551651, 1.2557525634765625, 1.5038087368011475, 1.482008934020996, 1.5400751829147339, 1.4717941284179688, 1.2966145277023315, 1.3099327087402344, 1.6249475479125977, 1.221555233001709, 1.9807698726654053, 1.5129830837249756, 1.272121548652649, 1.6036020517349243, 1.023245930671692, 1.25905179977417, 1.2711199522018433, 1.5763155221939087, 1.220418930053711, 1.1096885204315186, 1.4605798721313477, 1.2443971633911133, 1.5927339792251587, 1.2885501384735107, 1.6739444732666016, 1.4085699319839478, 1.5774919986724854, 1.3565908670425415, 1.6697018146514893, 1.4158105850219727, 1.181951642036438, 1.4757001399993896, 1.2253402471542358, 1.7004435062408447, 1.3363028764724731, 1.22817063331604, 1.2001922130584717, 1.406693458557129, 1.2779474258422852, 1.5431026220321655, 1.4543160200119019, 1.7543805837631226, 1.4680675268173218, 1.5329786539077759, 1.429962158203125, 1.4847153425216675, 1.3552197217941284, 1.6521382331848145, 1.1657603979110718, 1.810097098350525, 1.1719789505004883, 0.6925684213638306, 1.661705493927002, 1.2585639953613281, 0.8022041320800781, 1.8054134845733643, 0.9463993310928345, 2.145601749420166, 2.0960283279418945, 1.2761468887329102, 1.521202802658081, 1.649282693862915, 1.7744148969650269, 1.1677703857421875, 1.852217674255371, 1.9961049556732178, 1.9133480787277222, 1.7700254917144775, 1.2078194618225098, 1.5386388301849365, 1.7480729818344116, 1.9808096885681152, 2.0897507667541504, 1.7018717527389526, 1.7866863012313843, 1.3006285429000854, 1.26396644115448, 2.114607095718384, 1.566582202911377, 1.2870826721191406, 0.7315911054611206, 1.6440726518630981, 1.5689091682434082, 1.1346770524978638, 0.778125524520874, 2.2206218242645264, 1.1798796653747559, 1.169907569885254, 1.780005693435669, 0.8302398920059204, 1.32426118850708, 1.3001165390014648, 1.5431122779846191, 1.530057668685913, 0.7362111210823059, 1.6268103122711182, 0.8462206125259399, 1.1480273008346558, 1.1624583005905151, 1.663691520690918, 1.2603421211242676, 1.820486307144165, 1.0636616945266724, 2.392331600189209, 1.7976186275482178, 1.6705758571624756, 1.9325636625289917, 1.443497896194458, 1.043187141418457, 2.049922466278076, 1.267655372619629, 1.4649558067321777, 1.1199129819869995, 1.717496395111084, 1.855196237564087, 1.0996460914611816, 1.151823878288269, 1.2186716794967651, 1.8508687019348145, 1.1174755096435547, 1.4120572805404663, 1.290905475616455, 0.9771343469619751, 1.787028431892395, 1.2553040981292725, 1.731502652168274, 0.6568804383277893, 1.4374852180480957, 1.217595100402832, 2.3334245681762695, 1.576305866241455, 1.0441827774047852, 1.8305522203445435, 0.9945408701896667, 1.7962974309921265, 0.9487859606742859, 1.4979166984558105, 1.999751329421997, 1.5073350667953491, 2.606968402862549, 1.5726521015167236, 1.7945632934570312, 0.8829134702682495, 1.4722959995269775, 1.4441697597503662, 1.5918940305709839, 1.4032459259033203, 2.0150837898254395, 0.8649880886077881]
import matplotlib.pyplot as plt
# x axis values
x = []
for i in range(0,len(loss)):
    x.append(i)

plt.plot(x, loss)